### 1.0 import libraries

In [1]:
import os
import random

import pyvista as pv
import trimesh as tm
import numpy as np
import topogenesis as tg

### 1.1 load macrovoxels

In [2]:
# load lattice CSV file
lattice_path = os.path.relpath('../data/macrovoxels.csv')
macro_lattice = tg.lattice_from_csv(lattice_path)

### 1.x generate a single random configuration to be tested

In [3]:
# create random configuration
# config_rand = [random.randint(0,1) for i in range(27)]
config_rand = [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0]
print(config_rand)
# voxels of the configuration
config_index = [i for i in range(26) if config_rand[i] == 0] # random indices

config_rand = np.reshape(config_rand,(3,3,3))
config_rand = np.pad(config_rand, 1) # pad to regain original lattice structure


[0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0]


In [4]:
# saving the random configuration for future use
random_lattice = tg.to_lattice(config_rand, macro_lattice)
save_path = os.path.relpath("../data/random_lattice.csv")
random_lattice.to_csv(save_path)

## Identifying the roof

In [5]:
# shifting all the voxels one level sideways --> -Y direction
#shifted_lattice_Y_neg = np.roll(random_lattice_padded, (0,0,-1),axis=1) 

In [6]:
# shifting all the voxels one level sideways --> +X direction
#shifted_lattice_X_pos = np.roll(random_lattice_padded, (0,0,1),axis=2) 

In [7]:
# shifting all the voxels one level sideways --> -X direction
#shifted_lattice_X_neg = np.roll(random_lattice_padded, (0,0,-1),axis=2) 

In [8]:
# # padding to avoid the rolling issue
# random_lattice_padded = np.pad(random_lattice, 1, mode='constant',constant_values=0)

# # shifting all the voxels one level sideways --> +Y direction
# # TODO: unify into single function for each meshing approach from here to mesh list
# shifted_lattice_Y_pos = np.roll(random_lattice_padded, (0,0,1),axis=1) 

# # an exposed facade surface exists where a voxel is filled (1) and the voxel next to it is empty (0)
# side_voxels_3darray_padded = (random_lattice_padded == 1) *  (shifted_lattice_Y_pos == 0)

# # removing the pad
# side_voxels_3darray = side_voxels_3darray_padded[1:-1, 1:-1, 1:-1]
# # convert to lattice
# side_voxels_lattice = tg.to_lattice(side_voxels_3darray, random_lattice)

# # extracting the centroids of all side voxels
# side_centroids = side_voxels_lattice.centroids

In [9]:
def find_centroids(lattice, dir, axes):

    # shifting all the voxels one level sideways --> +Y direction
    shifted_lattice_Y_pos = np.roll(lattice, (0,0,dir),axis=axes) 
    
    # an exposed facade surface exists where a voxel is filled (1) and the voxel next to it is empty (0)
    side_voxels_3darray_padded = (lattice == 1) *  (shifted_lattice_Y_pos == 0)

    # removing the pad
    side_voxels_3darray = side_voxels_3darray_padded[1:-1, 1:-1, 1:-1]
    
    # convert to lattice
    side_voxels_lattice = tg.to_lattice(side_voxels_3darray, random_lattice)

    # extracting the centroids of all side voxels
    side_centroids = side_voxels_lattice.centroids

    return side_centroids

In [181]:
# Function for creating meshes

def construct_mesh_y_pos(centroid):
    meshes= []
    for i, cen in enumerate(centroid):
        # generating the vertices of the side faces in +Y direction
        # centroid + half of the unit size in the four top directions
        v0 = cen + 0.5 * unit * np.array([ 1, -1, 1]) # side right above
        v1 = cen + 0.5 * unit * np.array([ 1,-1, -1]) # side right below
        v2 = cen + 0.5 * unit * np.array([-1,-1, -1]) # side left below
        v3 = cen + 0.5 * unit * np.array([-1, -1, 1]) # side left above

        face_a = [v0,v1,v2] # trimesh only takes triangular meshes, no quad meshes
        face_b = [v2,v3,v0]

        mesh_a = tm.Trimesh(vertices= face_a, faces= [[2,1,0]])
        mesh_b = tm.Trimesh(vertices= face_b, faces= [[2,1,0]])

        meshes.append(mesh_a)
        meshes.append(mesh_b)

    return meshes

In [182]:
# Function for creating meshes
# TODO: adapt
def construct_mesh_y_neg(centroid):
    meshes= []
    for i, cen in enumerate(centroid):
        # generating the vertices of the side faces in +Y direction
        # centroid + half of the unit size in the four top directions
        v0 = cen + 0.5 * unit * np.array([ 1, 1, 1]) # side right above
        v1 = cen + 0.5 * unit * np.array([ 1,1, -1]) # side right below
        v2 = cen + 0.5 * unit * np.array([-1,1, -1]) # side left below
        v3 = cen + 0.5 * unit * np.array([-1, 1, 1]) # side left above

        face_a = [v0,v1,v2] # trimesh only takes triangular meshes, no quad meshes
        face_b = [v2,v3,v0]

        mesh_a = tm.Trimesh(vertices= face_a, faces= [[0,1,2]])
        mesh_b = tm.Trimesh(vertices= face_b, faces= [[0,1,2]])

        meshes.append(mesh_a)
        meshes.append(mesh_b)

    return meshes

In [205]:
# Function for creating meshes
# TODO: adapt
def construct_mesh_x_pos(centroid):
    meshes= []
    for i, cen in enumerate(centroid):
        # generating the vertices of the side faces in +Y direction
        # centroid + half of the unit size in the four top directions
        v0 = cen + 0.5 * unit * np.array([ -1, -1, 1]) # side right above
        v1 = cen + 0.5 * unit * np.array([ -1,-1, -1]) # side right below
        v2 = cen + 0.5 * unit * np.array([-1,1, -1]) # side left below
        v3 = cen + 0.5 * unit * np.array([-1, 1, 1]) # side left above

        face_a = [v0,v1,v2] # trimesh only takes triangular meshes, no quad meshes
        face_b = [v2,v3,v0]

        mesh_a = tm.Trimesh(vertices= face_a, faces= [[2,1,0]])
        mesh_b = tm.Trimesh(vertices= face_b, faces= [[2,1,0]])

        meshes.append(mesh_a)
        meshes.append(mesh_b)

    return meshes

In [216]:
# Function for creating meshes
# TODO: adapt
def construct_mesh_x_neg(centroid):
    meshes= []
    for i, cen in enumerate(centroid):
        # generating the vertices of the side faces in +Y direction
        # centroid + half of the unit size in the four top directions
        v0 = cen + 0.5 * unit * np.array([ 1, 1, 1]) # side right above
        v1 = cen + 0.5 * unit * np.array([1,1, -1]) # side right below
        v2 = cen + 0.5 * unit * np.array([1,-1, -1]) # side left below
        v3 = cen + 0.5 * unit * np.array([1, -1, 1]) # side left above

        face_a = [v0,v1,v2] # trimesh only takes triangular meshes, no quad meshes
        face_b = [v2,v3,v0]

        mesh_a = tm.Trimesh(vertices= face_a, faces= [[2,1,0]])
        mesh_b = tm.Trimesh(vertices= face_b, faces= [[2,1,0]])

        meshes.append(mesh_a)
        meshes.append(mesh_b)

    return meshes

In [217]:
def construct_vertical_mesh(lat):
    vertical_meshes = []

    # padding to avoid the rolling issue
    input_lattice_padded = np.pad(lat, 1, mode='constant',constant_values=0)

    # Y_positive
    Y_pos_centroids = find_centroids(lattice= input_lattice_padded, dir= 1, axes= 1)
    Y_pos_mesh = construct_mesh_y_pos(Y_pos_centroids)
    vertical_meshes.extend(Y_pos_mesh)

    # Y_negative
    Y_neg_centroids = find_centroids(lattice= input_lattice_padded, dir= -1, axes= 1)
    Y_neg_mesh = construct_mesh_y_neg(Y_neg_centroids)
    vertical_meshes.extend(Y_neg_mesh)

    # X_positive
    X_pos_centroids = find_centroids(lattice= input_lattice_padded, dir= 1, axes= 0)
    X_pos_mesh = construct_mesh_x_pos(X_pos_centroids)
    vertical_meshes.extend(X_pos_mesh)

    # X_negative
    X_neg_centroids = find_centroids(lattice= input_lattice_padded, dir= -1, axes= 0)
    X_neg_mesh = construct_mesh_x_neg(X_neg_centroids)
    vertical_meshes.extend(X_neg_mesh)
    
    return vertical_meshes

In [218]:
unit = random_lattice.unit

In [219]:
facade_mesh = tm.util.concatenate(construct_vertical_mesh(random_lattice))
norms = facade_mesh.face_normals
# TODO: check normals direction, formalize THIS^
len(norms)

32

In [220]:
vert = facade_mesh.vertices
count = 0
centroids = []
for i in range(len(norms)):
    value = 0
    for j in range(3):
        value += vert[3*i+j]
        value = value.astype(float)
        #print(value/3)
    centroids.append(value)



### Merge meshes

In [221]:
# combined roof mesh
#combined_roof = tm.util.concatenate(construct_mesh_y_pos(side_centroids))

In [222]:
# left = construct_mesh_y_pos(find_centroids(lattice= np.pad(random_lattice, 1, mode='constant',constant_values=0), dir= 1, axes= 1))
# right = construct_mesh_y_neg(find_centroids(lattice= np.pad(random_lattice, 1, mode='constant',constant_values=0), dir= -1, axes= 1))
# front = construct_mesh_x_pos(find_centroids(lattice= np.pad(random_lattice, 1, mode='constant',constant_values=0), dir= 1, axes= 0))
# back = construct_mesh_x_neg(find_centroids(lattice= np.pad(random_lattice, 1, mode='constant',constant_values=0), dir= -1, axes= 0))

In [223]:
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

In [224]:
len(centroids)


32

### 1.x visuzalizing

In [225]:
p = pv.Plotter(notebook=True)

p.add_mesh(tri_to_pv(facade_mesh), color='#abd8ff', opacity=0.99)
random_lattice.fast_vis(p,False,False,opacity=0.1)
p.add_arrows(np.array(centroids)/3,norms,mag=5)
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(153.81588256357912, 33.81588256357914, 93.81588256357914),
 (60.0, -60.0, 0.0),
 (0.0, 0.0, 1.0)]